# Extracting Corresponding Genius Data for Songs

Using the LyricsGenius [wrapper](https://github.com/johnwmillr/LyricsGenius) to access the Genius API in order to retreive songwriting details for each song utilized in this study

In [ ]:
# !pip install lyricsgenius
# !pip install psycopg2

In [38]:
import json
import lyricsgenius as genius
import numpy as np
import pandas as pd
import pickle
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
import time

In [4]:
api = genius.Genius('b9cxICX3BUl-Ioj6v2O9tX-RoLTvsZ5V1LSrWIMSAV0CBDDP7r86oXqX8wbXfXgT')

##### Example Submissions

In [5]:
example = api.search_song("Tearin' up My Heart - Radio Edit", artist_name='N SYNC')

Searching for "Tearin' up My Heart - Radio Edit" by N SYNC...
Done.


In [30]:
example = api.search_artist('Weird Al Yankovic', max_songs=2)

Searching for songs by Weird Al Yankovic...

Song 1: "Achy Breaky Song"
Song 2: "A Complicated Song"

Reached user-specified song limit (2).
Found 2 songs.
Done.


In [34]:
example.songs[1]

('A Complicated Song', '"Weird Al" Yankovic')

#### Retreiving Spotipy Song List

Given the Genius API, retreiving all of the tracks that I need via a song search is going to be too difficult. Therefore, I am going to retreive all songs per artist that I've searched.

#### Retreiving Spotipy Artist List

In [8]:
with open('../pickle/artist_list_td.pkl', 'rb+') as f:
    artist_list = pickle.load(f)

In [9]:
artist_list[:5]

["'N SYNC", '"WEIRD AL" YANKOVIC', '10 YEARS', '10,000 MANIACS', '112']

In [14]:
df = pd.read_csv('../data/spotify_artists_clean.csv', index_col=0)

In [15]:
df.head()

,external_urls,genres,href,id,images,name,popularity,type,uri,followers_2
0,{'spotify': 'https://open.spotify.com/artist/6...,"['boy band', 'dance pop', 'europop', 'pop', 'p...",https://api.spotify.com/v1/artists/6Ff53KvcvAj...,6Ff53KvcvAj5U7Z1vojB5o,"[{'height': 1173, 'url': 'https://i.scdn.co/im...",*NSYNC,71,artist,spotify:artist:6Ff53KvcvAj5U7Z1vojB5o,747136
1,{'spotify': 'https://open.spotify.com/artist/1...,"['antiviral pop', 'comedy rock', 'comic']",https://api.spotify.com/v1/artists/1bDWGdIC2ha...,1bDWGdIC2hardyt55nlQgG,"[{'height': 563, 'url': 'https://i.scdn.co/ima...","""Weird Al"" Yankovic",59,artist,spotify:artist:1bDWGdIC2hardyt55nlQgG,337751
2,{'spotify': 'https://open.spotify.com/artist/0...,"['alternative metal', 'nu metal', 'post-grunge...",https://api.spotify.com/v1/artists/0REMf7H0VP6...,0REMf7H0VP6DwfZ9MbuWph,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",10 Years,60,artist,spotify:artist:0REMf7H0VP6DwfZ9MbuWph,243895
3,{'spotify': 'https://open.spotify.com/artist/0...,"['alternative rock', 'folk', 'folk-pop', 'lili...",https://api.spotify.com/v1/artists/0MBIKH9DjtB...,0MBIKH9DjtBkv8O3nS6szj,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","10,000 Maniacs",52,artist,spotify:artist:0MBIKH9DjtBkv8O3nS6szj,108829
4,{'spotify': 'https://open.spotify.com/artist/7...,"['boy band', 'dance pop', 'gangster rap', 'hip...",https://api.spotify.com/v1/artists/7urq0VfqxEY...,7urq0VfqxEYEEiZUkebXT4,"[{'height': 130, 'url': 'https://i.scdn.co/ima...",112,68,artist,spotify:artist:7urq0VfqxEYEEiZUkebXT4,455231


In [ ]:
sw_list = []

for artist in df['name']:
    try:
        songs = api.search_artist(artist)
        i = 0
        for i in songs.songs[i]:
            try:
                sw_list.append((sw_list.songs.songs[i],
                               songs.songs[i].writer_artists))
            except:
                sw_list.append((artist, 'song not found'))
        time.sleep(0.5)
    except:
        sw_list.append((artist, 'songs not found'))

#### Connecting to postgres Database

In [25]:
def con_cur_to_db(dbname='postgres'):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host="54.218.249.164",
                  dbname="postgres",
                  user="postgres",
                  password="glide-mortuary-pod-cloy-belong")
    cur = con.cursor()
    return con, cur

In [29]:
con, cur = con_cur_to_db(dbname='postgres')